In [1]:
def make_data_object_name(dataset_name, year, month, day, hour, realization, forecast_period):
    template_string = "prods_op_{}_{:02d}{:02d}{:02d}_{:02d}_{:02d}_{:03d}.nc"
    return template_string.format(dataset_name, year, month, day, hour, realization, forecast_period)

In [258]:
import itertools
import iris
import os
import numpy as np
import cartopy.crs as ccrs
from datetime import datetime, timedelta

iris.FUTURE.netcdf_promote = True
iris.FUTURE.netcdf_no_unlimited = True

class UKGridcellDataset():
    def __init__(self, filenames, scale_factor, params, altitude_file='surface_altitude.nc', root='../data/'):
        filenames.sort()
        self.filenames = filenames
        self.scale_factor = scale_factor
        self.root = root
        self.params = params
        self.altitude_file = altitude_file
        times = [d for f in self.filenames for d in self._expand_date(self._extract_date(f))]
        self.times = [t for t in times if self._get_filename(t)[0] in self.filenames]
        self.n_times = len(self.times)
        
        c = iris.load(root+paths[0])[0]
        self.n_lats = c.coord('grid_latitude').shape[0]
        self.n_lons = c.coord('grid_longitude').shape[0]

    def _expand_date(self, d):
        hs = [i for i in range(1, 4)]
        if d.hour == 3:
            hs.append(0)
        return [d - timedelta(hours=h) for h in hs]

    def _extract_date(self, filename):
        t = datetime.strptime(filename[:31], 'prods_op_mogreps-uk_%Y%m%d_%H')
        lead_time = timedelta(hours=int(filename[-6:-3]))
        return t + lead_time
    
    def _reduce_dim(self, cube, dim):
        new_dim = np.linspace(cube.coord(dim).points[0], 
                              cube.coord(dim).points[-1],
                              num = cube.coord(dim).points.shape[0] // self.scale_factor)
        return new_dim
    
    def _upscale(self, cube):
        new = cube.copy()
        new_lat = self._reduce_dim(cube, 'grid_latitude')
        new_lon = self._reduce_dim(cube, 'grid_longitude')
        return new.interpolate(sample_points=[('grid_latitude', new_lat), ('grid_longitude', new_lon)],
                               scheme=iris.analysis.Linear())
    
    def _bilinear_downscale(self, upscaled, target):
        return upscaled.regrid(target, iris.analysis.Linear()) # defaults to n-linear

    def _get_filename(self, time):
        run = time
        while run.hour not in [3, 9, 15, 21]:
            run -= timedelta(hours=1)
            
        lead = time - run
        leadh = int(lead.total_seconds() / 3600)
        d_string = datetime.strftime(run, 'prods_op_mogreps-uk_%Y%m%d')
        fname = d_string + "_{:02d}_00_{:03d}.nc".format(run.hour, ((leadh // 3) + 1) * 3)
        
        return (fname, leadh % 3)
    
    def _select(self, c, **kwargs):
        s = [slice(None, None, None) for _ in range(c.ndim)]
        coords = [dc.standard_name for dc in c.dim_coords]
        for key, value in kwargs.items():
            s[c.coord_dims(key)[0]] = value
        return c[tuple(s)].data
        
    def _load_cell(self, time, lat, lon):
        result = {}
        filename, leadtime = self._get_filename(self.times[time])
        cubes = iris.load(self.root + filename, 
                          iris.AttributeConstraint(STASH=[p['stash'] for p in self.params]))

        crs = cubes[0].coords('grid_latitude')[0].coord_system.as_cartopy_crs()
        p_lat = cubes[0].coord('grid_latitude')[lat].points
        p_lon = cubes[0].coord('grid_longitude')[lon].points
        r_lon, r_lat = ccrs.PlateCarree().transform_point(p_lon, p_lat, crs)
        result['longitude'] = r_lon; result['latitude'] = r_lat;
        
        result['DOY'] = self.times[time].timetuple().tm_yday
        alt = iris.load(self.root + self.altitude_file)[0]
        result['surface_altitude'] = self._select(alt, grid_latitude=lat, grid_longitude=lon)
        
        for p in self.params:
            c_orig = cubes.extract(iris.AttributeConstraint(STASH=p['stash']))[0]
            c = self._bilinear_downscale(self._upscale(c_orig), target=c_orig)
            result[p['name']] = self._select(c, grid_latitude=lat, grid_longitude=lon, time=leadtime)
        
        p = {'name': 'stratiform_rainfall_amount', 'stash': 'm01s04i201'}
        c_orig = cubes.extract(iris.AttributeConstraint(STASH=p['stash']))[0]
        c = self._bilinear_downscale(self._upscale(c_orig), target=c_orig)
        result[p['name']] = self._select(c, grid_latitude=lat, grid_longitude=lon, time=leadtime)
        result[p['name'] + '_up'] = self._select(c, grid_latitude=lat+1, grid_longitude=lon, time=leadtime)
        result[p['name'] + '_down'] = self._select(c, grid_latitude=lat-1, grid_longitude=lon, time=leadtime)
        result[p['name'] + '_left'] = self._select(c, grid_latitude=lat, grid_longitude=lon-1, time=leadtime)
        result[p['name'] + '_right'] = self._select(c, grid_latitude=lat, grid_longitude=lon+1, time=leadtime)
        
        return result
        
    def _convert_id(self, idx):
        time = idx // ((self.n_lats - 2) * (self.n_lons - 2))
        r = idx % ((self.n_lats - 2) * (self.n_lons - 2))
        lat = r // (self.n_lons - 2)
        lon = r % (self.n_lons - 2)
        return (time, lat + 1, lon + 1)

    def __len__(self):
        return (self.n_times * (self.n_lats - 2) * (self.n_lons - 2)) - 1
    
    def __getitem__(self, idx):
        return self._load_cell(*self._convert_id(idx))
    

In [259]:
path = '../data/'
paths = [f for f in os.listdir(path) if f[:5] == 'prods']

In [260]:
params = [{'name': 'stratiform_rainfall_amount', 'stash': 'm01s04i201'},
          {'name': 'air_temperature', 'stash': 'm01s03i236'},
          {'name': 'surface_air_pressure', 'stash': 'm01s00i409'},
          {'name': 'x_wind', 'stash': 'm01s03i225'},
          {'name': 'y_wind', 'stash': 'm01s03i226'},
          {'name': 'specific_humidity', 'stash': 'm01s03i237'}]

In [261]:
dataset = UKGridcellDataset(paths, params=params, scale_factor=1)
dataset2 = UKGridcellDataset(paths, params=params, scale_factor=2)
dataset4 = UKGridcellDataset(paths, params=params, scale_factor=4)
dataset8 = UKGridcellDataset(paths, params=params, scale_factor=8)

In [262]:
len(dataset)

98830367

In [263]:
dataset.n_times * (dataset.n_lats - 2) * (dataset.n_lons - 2)

98830368

In [264]:
len(dataset) / ((dataset.n_lats - 2) * (dataset.n_lons - 2))

431.99999562887393

In [265]:
import pprint
# i = np.random.randint(len(dataset))
i = 421
pprint.pprint(dataset[i], width=1)
# pprint.pprint(dataset2[i], width=1)
# pprint.pprint(dataset4[i], width=1)
# pprint.pprint(dataset8[i], width=1)

{'DOY': 183,
 'air_temperature': array(287.5, dtype=float32),
 'latitude': 48.49583777312858,
 'longitude': -10.085604052623427,
 'specific_humidity': array(0.007568389177322388, dtype=float32),
 'stratiform_rainfall_amount': array(0.0, dtype=float32),
 'stratiform_rainfall_amount_down': array(0.0, dtype=float32),
 'stratiform_rainfall_amount_left': array(0.0, dtype=float32),
 'stratiform_rainfall_amount_right': array(0.0039036318194121122, dtype=float32),
 'stratiform_rainfall_amount_up': array(0.015624021179974079, dtype=float32),
 'surface_air_pressure': array(101518.0, dtype=float32),
 'surface_altitude': array(0.0, dtype=float32),
 'x_wind': array(7.624985218048096, dtype=float32),
 'y_wind': array(-3.7499845027923584, dtype=float32)}


In [266]:
c = iris.load(path + paths[0])

In [268]:
rain = c.extract('stratiform_rainfall_amount')[0]

In [309]:
rain = rain[0]

In [283]:
rain[0,20,20].data

array(0.265625, dtype=float32)

In [310]:
np.nonzero(rain.data)

(array([  0,   0,   0, ..., 547, 547, 547]),
 array([ 12,  13,  14, ..., 315, 316, 317]))

In [301]:
np.dstack(np.nonzero(rain[0].data))[0]

array([20, 20])

In [286]:
from sklearn.neighbors import NearestNeighbors

In [304]:
nbrs = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(np.dstack(np.where(rain.data == 0))[0])

In [305]:
nbrs.kneighbors(np.array([[20,20]]))

(array([[ 4.]]), array([[2736]]))

In [302]:
np.nonzero?

In [308]:
np.dstack(np.where(rain.data == 0))[0]

array([[  0,   0],
       [  0,   1],
       [  0,   2],
       ..., 
       [547, 418],
       [547, 419],
       [547, 420]])

In [320]:
rain.coord_system()

RotatedGeogCS(37.5, 177.5, ellipsoid=GeogCS(6371229.0))

In [319]:
rain.coord('grid_latitude').points[np.nonzero(rain.data)[0]]

array([-3.7799499 , -3.7799499 , -3.7799499 , ...,  7.16004944,
        7.16004944,  7.16004944], dtype=float32)

In [321]:
ccrs.OSGB?